In [ ]:
!pip install fastai==2.7.10
!pip install transformers

import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from fastai.text.all import *
import fastai
import re

print(torch.__version__)
print(transformers.__version__)
print( fastai.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.1 MB/s 
     |████████████████████████████████| 182 kB 79.0 MB/s 
     |████████████████████████████████| 7.6 MB 57.1 MB/s 
1.12.1+cu113
4.24.0
2.7.10


In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
# text = '근육이 커지기 위해서는'
# input_ids = tokenizer.encode(text, return_tensors='pt')
# gen_ids = model.generate(input_ids,
#                            max_length=128,
#                            repetition_penalty=2.0,
#                            pad_token_id=tokenizer.pad_token_id,
#                            eos_token_id=tokenizer.eos_token_id,
#                            bos_token_id=tokenizer.bos_token_id,
#                            use_cache=True)
# generated = tokenizer.decode(gen_ids[0])
# print(generated)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
df_bunsik = pd.read_csv('/df_bunsik_new.csv')

In [ ]:
df_cafe = pd.read_csv('/df_cafe_new.csv')

In [ ]:
df_chicken = pd.read_csv('/df_chicken_new.csv')

In [ ]:
df_hansik = pd.read_csv('/df_hansik_new.csv')

In [ ]:
df_ilsik = pd.read_csv('/df_ilsik_new.csv')

In [ ]:
df_jokbo = pd.read_csv('/df_jokbo_new.csv')

In [ ]:
df_joongsik = pd.read_csv('/df_joongsik_new.csv')

In [ ]:
df_pizza = pd.read_csv('/df_pizza_new.csv')

In [ ]:
# review 한줄로 만들기
review_str = ''
for i in range(len(df_pizza)):
    review_str += df_pizza.new_review[i]
    review_str += '</s>'

# 리뷰 글자수 세기
str_list = []
for i in range(len(df_pizza.new_review)):
    str_list.append(len(df_pizza.new_review[i]))
    if len(df_pizza.new_review[i]) >= 1000:
        print(i)
str_list.sort(reverse=True)
print(f"seq_len: {str_list[0]}")


# 
import re
lines = review_str
lines=" ".join(lines.split())

lines=re.sub('"', ' ', lines)
lines=re.sub("'", ' ', lines)

seq_len: 316


In [ ]:
#model input output tokenizer
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

#split data
train=lines[:int(len(lines)*0.9)]
test=lines[int(len(lines)*0.9):]
splits = [[0],[1]]

#init dataloader
tls = TfmdLists([train,test], TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
batch,seq_len = 8,316
dls = tls.dataloaders(bs=batch, seq_len=seq_len)
# dls.show_batch(max_n=2)

In [ ]:
#gpt2 ouput is tuple, we need just one val
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]
        
        
df_pizza_learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

In [ ]:
df_pizza_learn.fit_one_cycle(5,1e-4)

epoch,train_loss,valid_loss,perplexity,time
0,2.147985,2.163420,8.700844,01:39
1,1.982827,2.175872,8.809867,01:39
2,1.772887,2.187972,8.917113,01:39
3,1.574703,2.190819,8.942530,01:39
4,1.458672,2.199042,9.016375,01:39


In [ ]:
df_pizza_learn.model.save_pretrained("/train_models/pizza")
tokenizer.save_pretrained("/train_models/pizza")

('/content/drive/MyDrive/Itstudy/mid_proj/train_models/pizza/tokenizer_config.json',
 '/content/drive/MyDrive/Itstudy/mid_proj/train_models/pizza/special_tokens_map.json',
 '/content/drive/MyDrive/Itstudy/mid_proj/train_models/pizza/tokenizer.json')

In [ ]:
prompt='맛1 / 양1 / 배달1 / 배달'
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = df_chicken_learn.model.generate(inp,
                           max_length=100,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True,
                           do_sample=True
                          ) 
sentence = tokenizer.decode(preds[0].cpu().numpy())
print(sentence)

맛1 / 양1 / 배달1 / 배달완전빠르고 친절합니다.</s>
